## Finding the closest stations to open spaces, food trucks, farmers markets, and murals

In [1]:
import pandas as pd
import json
import geopy.distance

In [2]:
# returns number of stations requested by 'k'
# there's a column called "NUM STATIONS" for each open space that gives the number of stations 
# that are actually within the distance 'd' 

In [3]:
# constants
k = 10 # k-nearest neighbors
d = 0.3 # maximum distance in km

In [4]:
# file locations
open_space_source = 'data/spaces/Open_Space.geojson'
stations_source = 'data/bluebikes/current_bluebikes_stations_boston.csv'
food_truck_source = 'data/spaces/food_truck_schedule.geojson'
farmers_market_source = 'data/spaces/winter_farmers_markets.csv'
murals_source = 'data/spaces/murals.csv'

In [5]:
stations = pd.read_csv(stations_source)

In [6]:
# getting average coordinate of each open space using max/min long/lat
def average_coord (i):
    num_coords = len(data['features'][i]['geometry']["coordinates"][0][0])
    min_lat = 1000
    max_lat = -1000
    min_long = 1000
    max_long = -1000
    for j in range(0, num_coords):
        lat = data['features'][i]['geometry']["coordinates"][0][0][j][0]
        long = data['features'][i]['geometry']["coordinates"][0][0][j][1]
        if lat < min_lat: 
            min_lat = lat
        if lat > max_lat:
            max_lat = lat
        if long < min_long: 
            min_long = long
        if long > max_long:
            max_long = long
    return [(min_lat+max_lat)/2, (min_long+max_long)/2]

## Open spaces

In [7]:
f = open(open_space_source)
data = json.load(f)

In [23]:
open_spaces = pd.DataFrame(columns=['OBJECTID', 'OS_ID', 'PARK_ID', 'Name', 'Type', 'Acres', 'Avg. Lat', 'Avg. Long'])

In [24]:
for i in range(0, len(data['features'])):
    oid = data['features'][i]['properties']['OBJECTID']
    osid = data['features'][i]['properties']['OS_ID']
    parkid = data['features'][i]['properties']['PARK_ID']
    name = data['features'][i]['properties']['SITE_NAME']
    space_type = data['features'][i]['properties']['TypeLong']
    if space_type == 'Parks, Playgrounds & Athletic Fields':
        if 'Playground' in name:
            space_type = 'Playgrounds'
        elif 'Field' in name:
            space_type = 'Athletic Fields'
        else:
            space_type = 'Parks'
    acres = data['features'][i]['properties']['ACRES']
    ll = average_coord(i)
    open_spaces.loc[len(open_spaces.index)] = [oid, osid, parkid, name, space_type, acres, ll[1], ll[0]]  


In [25]:
open_spaces

,OBJECTID,OS_ID,PARK_ID,Name,Type,Acres,Avg. Lat,Avg. Long
0,1,2000,None,Wilson Park,"Malls, Squares & Plazas",0.097184,42.340991,-71.145929
1,2,156,156,James H. Roberts Playground,Playgrounds,1.004136,42.359526,-71.125332
2,3,284,284,Smith Playground,Playgrounds,15.025247,42.365072,-71.131046
3,4,115,115,Fern Square,"Malls, Squares & Plazas",0.044509,42.359534,-71.133291
4,5,259,259,Ringer Playground,Playgrounds,10.263093,42.350456,-71.138129
...,...,...,...,...,...,...,...,...
529,530,7273,7273,Mahoney Park,Parks,0.381102,42.342008,-71.051382
530,531,2106,2106,Boundary I,Urban Wilds,6.925776,42.269348,-71.137990
531,1340,7275,7275,Sprague Pond,Urban Wilds,1.196202,42.233786,-71.138006
532,2541,7276,7276,Codman Square,"Malls, Squares & Plazas",0.350710,42.290418,-71.071082


In [26]:
# fixing duplicate names
counts = open_spaces['Name'].value_counts()
duplicate_ids = list(open_spaces.loc[open_spaces['Name'].isin(list(counts.loc[counts > 1].index))].index)
counter = 1
for n in range(0, len(duplicate_ids) - 1):
    open_spaces.loc[duplicate_ids[n], 'Name'] = f"{open_spaces.loc[duplicate_ids[n], 'Name']} {counter}"
    if open_spaces.loc[duplicate_ids[n], 'Name'][:-2] == open_spaces.loc[duplicate_ids[n+1], 'Name']:
        counter += 1
    else:
        counter = 1
open_spaces.loc[duplicate_ids[-1], 'Name'] = f"{open_spaces.loc[duplicate_ids[-1], 'Name']} {counter}"
open_spaces.loc[duplicate_ids]

,OBJECTID,OS_ID,PARK_ID,Name,Type,Acres,Avg. Lat,Avg. Long
382,383,178,178,Lincoln Square 1,"Malls, Squares & Plazas",0.054628,42.351612,-71.067861
389,390,179,179,Lincoln Square 2,"Malls, Squares & Plazas",0.216593,42.334658,-71.033808
393,394,3084,None,Rink Grounds 1,Parks,0.734344,42.374113,-71.066424
516,517,3109,None,Rink Grounds 2,Parks,2.525996,42.288451,-71.044820


In [27]:
open_spaces.to_csv('data/spaces/open_space_coords.csv', index=False)

In [13]:
meta_dists = []

for i in range(len(open_spaces)):
    space = (open_spaces["Avg. Lat"][i], open_spaces["Avg. Long"][i])
    dists = []

    for j in range(len(stations)):
        stat = (stations['Latitude'][j], stations['Longitude'][j])
        distance = geopy.distance.geodesic(space, stat).km
        dists += [(distance, stations['Number'][j])]

    meta_dists += [sorted(dists)[0:k]]

In [14]:
meta_dists

[[(0.48175020260611406, 'D32033'),
  (0.6451252195913203, 'D32054'),
  (0.7178978731245395, 'B32035'),
  (0.9536097884489997, 'D32028'),
  (0.954297150676388, 'D32034'),
  (1.4195919155401824, 'A32017'),
  (1.4791050371730843, 'D32029'),
  (1.5439523396166428, 'A32001'),
  (1.6042470013801102, 'D32035'),
  (1.7646484982898225, 'A32045')],
 [(0.2757887457209742, 'A32040'),
  (0.4690397022800918, 'A32011'),
  (0.6053261879248848, 'A32019'),
  (0.7841735712695247, 'A32006'),
  (0.8163720968811304, 'A32012'),
  (0.948058405005651, 'A32052'),
  (1.0130923935598353, 'A32002'),
  (1.0482694356956503, 'A32005'),
  (1.1812705058994295, 'D32035'),
  (1.2030185969675193, 'A32001')],
 [(0.22720176045357812, 'A32019'),
  (0.5522332663512377, 'A32011'),
  (0.5720422564623499, 'A32040'),
  (0.6520696225138722, 'A32052'),
  (0.6778668546111044, 'A32005'),
  (1.0558066040021403, 'A32006'),
  (1.2886252612554596, 'A32045'),
  (1.402350648912702, 'A32001'),
  (1.5422007120010606, 'A32012'),
  (1.55249487

In [15]:
meta_dists_saved = meta_dists

In [16]:
open_spaces_nearest_stations = pd.DataFrame(columns=['OBJECTID', 'OS_ID', 'PARK_ID', 'Name', 'Avg. Lat', 'Avg. Long', 'NUM STATIONS'])


for i in range(k):
    open_spaces_nearest_stations[("Station {}".format(str(i+1)))] = []
    open_spaces_nearest_stations[("Station {} Dist ".format(str(i+1)))] = []

In [17]:
for i in range(len(meta_dists)):
    
    
    
    oid = open_spaces['OBJECTID'][i]
    osid = open_spaces['OS_ID'][i]
    parkid = open_spaces['PARK_ID'][i]
    name = open_spaces['Name'][i]
    lat = open_spaces['Avg. Lat'][i]
    long = open_spaces['Avg. Long'][i]
    
    
    to_return = [oid, osid, parkid, name, lat, long] 
    to_return_2 = []
    
    num = 0
    
    for j in range(k):
        stat = meta_dists[i][j][1]
        stat_d = meta_dists[i][j][0]
        
        if stat_d < d:
            num += 1
        
        to_return_2 += [stat, stat_d]
        
    
    to_return_2 = [num] + to_return_2

    to_return += to_return_2
    
    open_spaces_nearest_stations.loc[len(open_spaces_nearest_stations.index)] = to_return


In [18]:
open_spaces_nearest_stations

,OBJECTID,OS_ID,PARK_ID,Name,Avg. Lat,Avg. Long,NUM STATIONS,Station 1,Station 1 Dist,Station 2,...,Station 6,Station 6 Dist,Station 7,Station 7 Dist,Station 8,Station 8 Dist,Station 9,Station 9 Dist,Station 10,Station 10 Dist
0,1,2000,None,Wilson Park,42.340991,-71.145929,0,D32033,0.481750,D32054,...,A32017,1.419592,D32029,1.479105,A32001,1.543952,D32035,1.604247,A32045,1.764648
1,2,156,156,James H. Roberts Playground,42.359526,-71.125332,1,A32040,0.275789,A32011,...,A32052,0.948058,A32002,1.013092,A32005,1.048269,D32035,1.181271,A32001,1.203019
2,3,284,284,Smith Playground,42.365072,-71.131046,1,A32019,0.227202,A32011,...,A32006,1.055807,A32045,1.288625,A32001,1.402351,A32012,1.542201,D32001,1.552495
3,4,115,115,Fern Square,42.359534,-71.133291,0,A32040,0.401155,A32005,...,A32011,0.853399,A32012,1.122541,D32001,1.134382,D32035,1.139529,A32017,1.146023
4,5,259,259,Ringer Playground,42.350456,-71.138129,1,D32034,0.278138,A32001,...,A32045,0.738375,D32028,1.014448,D32001,1.041946,A32012,1.194989,A32005,1.232182
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
529,530,7273,7273,Mahoney Park,42.342008,-71.051382,1,C32024,0.257203,C32029,...,C32016,0.794830,D32018,0.809313,C32035,0.916724,C32091,0.948956,C32025,1.064587
530,531,2106,2106,Boundary I,42.269348,-71.137990,0,C32106,1.553058,E32013,...,C32079,2.034329,C32046,2.049453,C32047,2.142205,C32086,2.181824,C32085,2.277101
531,1340,7275,7275,Sprague Pond,42.233786,-71.138006,0,C32104,2.669196,E32013,...,C32045,5.269233,C32087,5.287377,C32079,5.296514,C32086,5.669837,C32044,5.824563
532,2541,7276,7276,Codman Square,42.290418,-71.071082,1,C32038,0.060420,C32055,...,C32062,0.776719,C32041,1.335041,C32043,1.400501,C23109,1.498176,C32103,1.502327


In [19]:
open_spaces_nearest_stations.to_csv('open_spaces_nearest_stations_k_{}_d_{}.csv'.format(str(k), str(d)), index=False)

## Food trucks

In [20]:
f = open(food_truck_source)
data = json.load(f)

In [21]:
food_trucks = pd.DataFrame(columns=['OBJECTID', 'Name', 'Lat', 'Long'])

In [22]:
data['features'][0]['properties']['ObjectId']

1

In [23]:
for i in range(len(data['features'])):
    oid = data['features'][i]['properties']['ObjectId']
    name = data['features'][i]['properties']['Truck']
    lat = data['features'][i]['properties']['y']
    long = data['features'][i]['properties']['x']
    food_trucks.loc[len(food_trucks.index)] = [oid, name, lat, long]

In [24]:
food_trucks

,OBJECTID,Name,Lat,Long
0,1,Indian Street Cravings,42.350534,-71.075124
1,2,Moyzilla,42.350534,-71.075124
2,3,Hungry Nomads,42.350534,-71.075124
3,4,Chicken and Rice Guys,42.350534,-71.075124
4,5,Tacos Calleteco,42.350534,-71.075124
5,6,Hungry Nomads,42.350534,-71.075124
6,7,Indian Street Cravings,42.360581,-71.057741
7,8,Chick-fil-A,42.377834,-71.051037
8,9,Chick-fil-A,42.377834,-71.051037
9,10,Indian Street Cravings,42.340478,-71.088974


In [25]:
meta_dists = []

for i in range(len(food_trucks)):
    truck = (food_trucks["Lat"][i], food_trucks["Long"][i])
    dists = []

    for j in range(len(stations)):
        stat = (stations['Latitude'][j], stations['Longitude'][j])
        distance = geopy.distance.geodesic(truck, stat).km
        dists += [(distance, stations['Number'][j])]

    meta_dists += [sorted(dists)[0:k]]

In [26]:
food_trucks_nearest_stations = pd.DataFrame(columns=['OBJECTID', 'Name', 'Avg. Lat', 'Avg. Long', 'NUM STATIONS'])


for i in range(k):
    food_trucks_nearest_stations[("Station {}".format(str(i+1)))] = []
    food_trucks_nearest_stations[("Station {} Dist ".format(str(i+1)))] = []

In [27]:

for i in range(len(meta_dists)):
    
    oid = food_trucks['OBJECTID'][i]
    name = food_trucks['Name'][i]
    lat = food_trucks['Lat'][i]
    long = food_trucks['Long'][i]  
    
    
    
    to_return = [oid, name, lat, long] 
    to_return_2 = []
    
    num = 0
    
    for j in range(k):
        stat = meta_dists[i][j][1]
        stat_d = meta_dists[i][j][0]
        
        if stat_d < d:
            num += 1
        
        to_return_2 += [stat, stat_d]
        
    
    to_return_2 = [num] + to_return_2

    to_return += to_return_2
    
    food_trucks_nearest_stations.loc[len(food_trucks_nearest_stations.index)] = to_return


In [28]:
food_trucks_nearest_stations.to_csv('food_trucks_nearest_stations_k_{}_d_{}.csv'.format(str(k), str(d)), index=False)

In [29]:
food_trucks_nearest_stations

,OBJECTID,Name,Avg. Lat,Avg. Long,NUM STATIONS,Station 1,Station 1 Dist,Station 2,Station 2 Dist,Station 3,...,Station 6,Station 6 Dist,Station 7,Station 7 Dist,Station 8,Station 8 Dist,Station 9,Station 9 Dist,Station 10,Station 10 Dist
0,1,Indian Street Cravings,42.350534,-71.075124,5,D32013,0.165324,D32005,0.198625,D32045,...,D32046,0.372973,D32007,0.419730,D32017,0.585890,D32042,0.619421,C32008,0.627141
1,2,Moyzilla,42.350534,-71.075124,5,D32013,0.165324,D32005,0.198625,D32045,...,D32046,0.372973,D32007,0.419730,D32017,0.585890,D32042,0.619421,C32008,0.627141
2,3,Hungry Nomads,42.350534,-71.075124,5,D32013,0.165324,D32005,0.198625,D32045,...,D32046,0.372973,D32007,0.419730,D32017,0.585890,D32042,0.619421,C32008,0.627141
3,4,Chicken and Rice Guys,42.350534,-71.075124,5,D32013,0.165324,D32005,0.198625,D32045,...,D32046,0.372973,D32007,0.419730,D32017,0.585890,D32042,0.619421,C32008,0.627141
4,5,Tacos Calleteco,42.350534,-71.075124,5,D32013,0.165324,D32005,0.198625,D32045,...,D32046,0.372973,D32007,0.419730,D32017,0.585890,D32042,0.619421,C32008,0.627141
5,6,Hungry Nomads,42.350534,-71.075124,5,D32013,0.165324,D32005,0.198625,D32045,...,D32046,0.372973,D32007,0.419730,D32017,0.585890,D32042,0.619421,C32008,0.627141
6,7,Indian Street Cravings,42.360581,-71.057741,5,D32009,0.025547,B32008,0.184724,B32032,...,B32061,0.378214,A32053,0.480502,A32013,0.532714,B32056,0.540391,B32004,0.601716
7,8,Chick-fil-A,42.377834,-71.051037,1,D32023,0.182586,D32039,0.516153,D32037,...,A32041,1.011394,D32021,1.011495,D32038,1.103983,D32020,1.144846,A32031,1.327899
8,9,Chick-fil-A,42.377834,-71.051037,1,D32023,0.182586,D32039,0.516153,D32037,...,A32041,1.011394,D32021,1.011495,D32038,1.103983,D32020,1.144846,A32031,1.327899
9,10,Indian Street Cravings,42.340478,-71.088974,1,B32062,0.190111,Z32997,0.372431,B32005,...,C32004,0.607343,D32061,0.618708,C32056,0.641831,B32021,0.681783,B32064,0.705161


## Farmers markets (winter only)

In [30]:
data = pd.read_csv(farmers_market_source)
data

,Market Name,Type,Address,Zip code,Latitude,Longitude,Neighborhood,Market days,Market Hours,First day of Market,Last day of Market,WIC and Senior Farmers Market Nutrition Program Vouchers,Accepts City of Boston Coupons
0,Brighton Farmers Market,In-Person,"640 Washington St, Brighton MA 02135",2135,42.347872,-71.153686,Brighton,"Every other Saturday: 01/27, 02/10, 02/24, 03/...",11am-2pm,01/27/24,03/30/24,WIC,NaN
1,Dorchester Winter Farmers Market,In-Person,"6 Norfolk St., Dorchester MA, 02124",2124,42.289960,-71.071840,Dorchester,Saturday,11am-2pm,01/27/23,03/30/24,"WIC, SFMNPV",NaN
2,East Boston Farmers Market,In-Person,"250 Sumner St, East Boston MA 02128",2128,42.375231,-71.040673,East Boston,Thursday,2pm-6pm,11/16/23,06/20/24,"WIC, SFMNPV",NaN
3,WeGrowMicrogreens,In-Person,"21 Norton St, Hyde Park MA 02136",2136,42.242854,-71.136693,Hyde Park,Friday,1pm-4:30pm,10/27/23,03/08/24,"WIC, SFMNPV",NaN
4,JP Centre St Farmers Market,In-Person,677 Centre St. Bank of America Parking Lot,2130,42.312173,-71.114370,Jamaica Plain,Saturday,12pm-3pm,11/11/24,03/30/24,"WIC, SFMNPV",NaN
5,Roslindale Farmers Market,In-Person,"Adams Park, Roslindale 02131",2131,42.286545,-71.128441,Roslindale,Sunday,11am-2pm,01/7/24,03/31/24,Neither,NaN
6,Dudley Town Common Farmers Market,In-Person,"11 Brook Ave, Roxbury MA 02119",2119,42.325115,-71.074876,Roxbury,Thursday,2:30pm-6:00pm,11/3/23,02/01/24,"WIC, SFMNPV",NaN


In [31]:
meta_dists = []

for i in range(len(data)):
    market = (data["Latitude"][i], data["Longitude"][i])
    dists = []

    for j in range(len(stations)):
        stat = (stations['Latitude'][j], stations['Longitude'][j])
        distance = geopy.distance.geodesic(market, stat).km
        dists += [(distance, stations['Number'][j])]

    meta_dists += [sorted(dists)[0:k]]

In [32]:
farmers_markets_nearest_stations = pd.DataFrame(columns=['OBJECTID', 'Name', 'Avg. Lat', 'Avg. Long', 'NUM STATIONS'])


for i in range(k):
    farmers_markets_nearest_stations[("Station {}".format(str(i+1)))] = []
    farmers_markets_nearest_stations[("Station {} Dist ".format(str(i+1)))] = []

In [33]:
for i in range(len(meta_dists)):
    
    name = data['Market Name'][i]
    lat = data['Latitude'][i]
    long = data['Longitude'][i]
    
    
    to_return = [i, name, lat, long] 
    to_return_2 = []
    
    num = 0
    
    for j in range(k):
        stat = meta_dists[i][j][1]
        stat_d = meta_dists[i][j][0]
        
        if stat_d < d:
            num += 1
        
        to_return_2 += [stat, stat_d]
        
    
    to_return_2 = [num] + to_return_2

    to_return += to_return_2
    
    farmers_markets_nearest_stations.loc[len(farmers_markets_nearest_stations.index)] = to_return


In [34]:
farmers_markets_nearest_stations

,OBJECTID,Name,Avg. Lat,Avg. Long,NUM STATIONS,Station 1,Station 1 Dist,Station 2,Station 2 Dist,Station 3,...,Station 6,Station 6 Dist,Station 7,Station 7 Dist,Station 8,Station 8 Dist,Station 9,Station 9 Dist,Station 10,Station 10 Dist
0,0,Brighton Farmers Market,42.347872,-71.153686,0,D32028,0.305716,D32029,0.559374,D32054,...,D32034,1.131310,D32001,1.196508,A32045,1.382370,B32035,1.392648,A32001,1.479239
1,1,Dorchester Winter Farmers Market,42.289960,-71.071840,2,C32038,0.041527,C32055,0.294878,C32110,...,C32062,0.792420,C32043,1.360089,C32041,1.414260,C32103,1.427107,C23109,1.496906
2,2,East Boston Farmers Market,42.375231,-71.040673,2,A32041,0.124877,A32036,0.215055,A32033,...,A32049,0.665416,A32030,0.713274,D32023,0.762405,A32044,1.027133,A32057,1.061510
3,3,WeGrowMicrogreens,42.242854,-71.136693,0,C32104,1.739842,E32013,1.944959,E32014,...,Z32999,4.423152,C32087,4.433841,C32045,4.512427,C32086,4.749278,C32059,4.831049
4,4,JP Centre St Farmers Market,42.312173,-71.114370,0,E32001,0.357360,E32005,0.605974,D32052,...,E32003,1.128959,E32010,1.249734,E32004,1.354909,C32102,1.500828,E32006,1.674453
5,5,Roslindale Farmers Market,42.286545,-71.128441,2,C32046,0.032757,C32047,0.081500,C32059,...,C32089,1.472931,C32105,1.545995,E32010,1.980068,C32085,2.062637,C32106,2.184864
6,6,Dudley Town Common Farmers Market,42.325115,-71.074876,1,C32015,0.046263,C32022,0.709960,B32017,...,C32005,1.160866,D32059,1.275472,B32055,1.349080,C32061,1.371186,B32023,1.411001


In [35]:
farmers_markets_nearest_stations.to_csv('farmers_markets_nearest_stations_k_{}_d_{}.csv'.format(str(k), str(d)), index=False)

## Murals

In [36]:
data = pd.read_csv(murals_source)
data

,Artist,Location,Coordinates,Location Notes,Title,Year,Description,Neighborhood
0,"Wen-ti Tsen, Yuon Zuo","Oxford St, Boston, MA 02111","42.35169668236009, -71.0605710631534",Pocket park off of Oxford Street,Chinatown Heritage,1988,The Chinatown Heritage Mural is a copy of a 17...,Chinatown
1,"Vyal ""Vyal One"" Reyes","Underground at Ink Block, 90 Traveler Street, ...","42.345023834047964, -71.06052905619565",NaN,Untitled,2017,"Vyal ""Vyal One"" Reyes usualy includes the styl...",South End
2,"Victor ""Marka27"" Quiñonez, Rob ""ProBlak"" Gibbs","Underground at Ink Block, 90 Traveler Street, ...","42.345023834047964, -71.06052905619565",NaN,Don’t Let Me Be Misunderstood,2020,"In summer 2020, local artists Rob ""ProBlak"" Gi...",South End
3,"Victor ""Marka27"" Quiñonez, David ""Don Rimx"" Se...","Maurice J Tobin School, 40 Smith St, Roxbury, ...","42.33310247231317, -71.0983383021578",NaN,Untitled (Tobin School),2018,One of the many collaborations adorning the To...,Roxbury
4,"Victor ""Marka27"" Quiñonez","Washington Manor, 1701 Washington St. South End","42.33754240652663, -71.07612459818394",Visible from Worcester St.,Souledad,2022,The City of Boston commissioned artist Victor ...,South End
...,...,...,...,...,...,...,...,...
136,Alex Cook,"Norfolk St. and Ferndale St., Dorchester, MA 0...","42.28828967921538, -71.07709872887105",On side of building facing Ferndale St.,Beholding the Holy City (Father and Daughter E...,2019,Alex Cook worked with youth from The Boston Pr...,Dorchester
137,Alex Cook,"Centre St. and Starr Lane, Jamaica Plain, MA 0...","42.313140374687976, -71.11411952887104",On side of building facing Starr Lane,Starr Lane Landscape,2004,This brightly colored mural depicts several ch...,Jamaica Plain
138,Alex Cook,"Whittier School Apartments, 60 Southern Ave, D...","42.29001046240485, -71.07489886442673",On wall of seating area behind building,Untitled,2021,Alex worked with youth from The Boston Project...,Dorchester
139,Alex Cook,"618 Shawmut Ave, Roxbury, MA 02118","42.33585029634547, -71.08057177943742",Facing Ramsay Park,She Sees More Than The World Will Show Her,2009,This mural by Alex Cook features a woman stand...,Roxbury


In [37]:
data[['Latitude', 'Longitude']] = data.apply(lambda x: x['Coordinates'].split(', '), axis=1, result_type='expand')

In [38]:
meta_dists = []

for i in range(len(data)):
    market = (data["Latitude"][i], data["Longitude"][i])
    dists = []

    for j in range(len(stations)):
        stat = (stations['Latitude'][j], stations['Longitude'][j])
        distance = geopy.distance.geodesic(market, stat).km
        dists += [(distance, stations['Number'][j])]

    meta_dists += [sorted(dists)[0:k]]

In [39]:
murals_nearest_stations = pd.DataFrame(columns=['OBJECTID', 'Title', 'Artist', 'Avg. Lat', 'Avg. Long', 'NUM STATIONS'])


for i in range(k):
    murals_nearest_stations[("Station {}".format(str(i+1)))] = []
    murals_nearest_stations[("Station {} Dist ".format(str(i+1)))] = []

In [40]:
murals_nearest_stations

,OBJECTID,Title,Artist,Avg. Lat,Avg. Long,NUM STATIONS,Station 1,Station 1 Dist,Station 2,Station 2 Dist,...,Station 6,Station 6 Dist,Station 7,Station 7 Dist,Station 8,Station 8 Dist,Station 9,Station 9 Dist,Station 10,Station 10 Dist


In [41]:
for i in range(len(meta_dists)):
    
    title = data['Title'][i]
    artist = data['Artist'][i]
    lat = data['Latitude'][i]
    long = data['Longitude'][i]
    
    
    to_return = [i, title, artist, lat, long] 
    to_return_2 = []
    
    num = 0
    
    for j in range(k):
        stat = meta_dists[i][j][1]
        stat_d = meta_dists[i][j][0]
        
        if stat_d < d:
            num += 1
        
        to_return_2 += [stat, stat_d]
        
    
    to_return_2 = [num] + to_return_2

    to_return += to_return_2
    
    murals_nearest_stations.loc[len(murals_nearest_stations.index)] = to_return


In [42]:
murals_nearest_stations

,OBJECTID,Title,Artist,Avg. Lat,Avg. Long,NUM STATIONS,Station 1,Station 1 Dist,Station 2,Station 2 Dist,...,Station 6,Station 6 Dist,Station 7,Station 7 Dist,Station 8,Station 8 Dist,Station 9,Station 9 Dist,Station 10,Station 10 Dist
0,0,Chinatown Heritage,"Wen-ti Tsen, Yuon Zuo",42.35169668236009,-71.0605710631534,3,D32015,0.106179,D32019,0.190853,...,D32014,0.430442,D32011,0.475074,D32048,0.541910,B32056,0.561224,D32042,0.596729
1,1,Untitled,"Vyal ""Vyal One"" Reyes",42.345023834047964,-71.06052905619565,4,C32035,0.200757,C32025,0.239713,...,D32043,0.505838,D32048,0.571885,C32024,0.660105,D32015,0.709865,C32000,0.749893
2,2,Don’t Let Me Be Misunderstood,"Victor ""Marka27"" Quiñonez, Rob ""ProBlak"" Gibbs",42.345023834047964,-71.06052905619565,4,C32035,0.200757,C32025,0.239713,...,D32043,0.505838,D32048,0.571885,C32024,0.660105,D32015,0.709865,C32000,0.749893
3,3,Untitled (Tobin School),"Victor ""Marka27"" Quiñonez, David ""Don Rimx"" Se...",42.33310247231317,-71.0983383021578,1,B32063,0.240612,C32001,0.336983,...,B32036,0.513104,B32021,0.526367,B32003,0.607191,Z32997,0.762036,B32055,0.786102
4,4,Souledad,"Victor ""Marka27"" Quiñonez",42.33754240652663,-71.07612459818394,1,C32006,0.202859,C32005,0.362362,...,D32061,0.529607,C32093,0.556446,C32060,0.577994,C32004,0.582097,C32061,0.600002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136,136,Beholding the Holy City (Father and Daughter E...,Alex Cook,42.28828967921538,-71.07709872887105,0,C32110,0.445928,C32038,0.491996,...,C32103,1.092103,C32062,1.107086,C32092,1.172994,C32066,1.257215,C32094,1.478068
137,137,Starr Lane Landscape,Alex Cook,42.313140374687976,-71.11411952887104,0,E32001,0.466593,E32005,0.627112,...,E32003,1.020894,E32004,1.307737,E32010,1.357139,C32102,1.482043,E32012,1.583231
138,138,Untitled,Alex Cook,42.29001046240485,-71.07489886442673,1,C32038,0.257582,C32110,0.355252,...,C32066,1.034249,C32043,1.127982,C32103,1.327692,C32092,1.399169,C32111,1.491983
139,139,She Sees More Than The World Will Show Her,Alex Cook,42.33585029634547,-71.08057177943742,3,D32059,0.015683,C32005,0.151484,...,C32004,0.560394,B32002,0.612567,B32064,0.613416,C32006,0.614228,C32063,0.653607


In [43]:
murals_nearest_stations.to_csv('murals_nearest_stations_k_{}_d_{}.csv'.format(str(k), str(d)), index=False)